In [96]:
## matrix for latency between servers
serverCount =7
matrix=[[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
for i in range(0,serverCount):
    for j in range(i+1,serverCount):
        x=random.randint(1,10)
        matrix[i][j]=x
        matrix[j][i]=x
    
matrix

[[0, 7, 2, 4, 7, 9, 1],
 [7, 0, 5, 7, 9, 2, 10],
 [2, 5, 0, 5, 4, 3, 5],
 [4, 7, 5, 0, 9, 4, 10],
 [7, 9, 4, 9, 0, 3, 3],
 [9, 2, 3, 4, 3, 0, 7],
 [1, 10, 5, 10, 3, 7, 0]]

In [97]:
def objectiveFunction(x):
    total_latency=0
    ## fetch top 3 probailities for vnf placemnent
    firstmax=-1
    firstvnf=-1
    secondmax=-1
    secondvnf=-1
    thirdmax=-1
    thirdvnf=-1
    possibilities=[[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]
    j=0
    for i in x:
        if i>firstmax:
            firstmax=i
            firstvnf=j
        elif i>secondmax:
            secondmax=i
            secondvnf=j
        elif i>thirdmax:
            thirdmax=i
            thirdvnf=j
        j=j+1
    total_latency=matrix[firstvnf][secondvnf]+matrix[secondvnf][thirdvnf]
    print(total_latency)
    return total_latency

## 1 2 3
## 1 3 2
## 2 1 3
## 2 3 1
## 3 1 2
## 3 2 1

In [98]:
class Particle:
    def __init__(self):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual inshort it is best latency
        self.err_i=-1               # error individual
        
        
        for i in range(0,num_dimensions):      # intializing position and velocity
            self.position_i.append(random.uniform(0,1))
            self.velocity_i.append(random.uniform(-1,1))
            
            
    # evaluate current fitness
    def evaluate(self,objectiveFunction):
        self.err_i=objectiveFunction(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
        
    
            
     # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
                       
            # adjust maximum position if necessary
           # if self.position_i[i]>bounds[1]:
           #     self.position_i[i]=bounds[1]

            # adjust minimum position if neseccary
            #if self.position_i[i] < bounds[0]:
             #   self.position_i[i]=bounds[0]

     # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.7         # inertia weight 
        c1=1.5        # congative constant
        c2=1.5        # social constant
        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

In [99]:
class PSO:
    def __init__(self,objectiveFunction,serverCount,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=serverCount
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle())

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(objectiveFunction)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print (pos_best_g)
        print (err_best_g)

if __name__ == "__PSO__":
    main()


In [100]:
bounds=[0,1]
PSO(objectiveFunction,serverCount,bounds,num_particles=15,maxiter=50)

10
6
13
9
6
12
11
12
5
10
6
12
6
11
13
8
10
9
6
14
10
8
7
14
10
12
10
12
6
15
19
8
5
6
6
14
7
12
5
9
12
10
12
6
12
10
12
12
6
7
5
12
12
14
12
12
5
8
6
14
12
6
6
5
10
10
14
6
8
12
5
12
19
14
5
9
10
14
6
12
10
12
10
14
14
6
10
8
11
12
7
3
10
10
12
10
12
10
12
6
14
12
7
11
12
10
9
10
10
3
6
12
10
6
10
9
12
13
19
7
17
9
7
10
13
13
6
10
3
6
10
7
3
13
9
10
13
9
10
9
3
13
3
3
10
13
7
10
10
10
10
13
10
7
9
13
3
10
10
9
10
7
10
8
6
13
13
10
10
12
13
12
12
8
9
10
13
3
7
14
13
7
6
13
12
13
12
10
3
10
7
10
3
10
10
7
7
13
13
15
3
10
10
13
13
3
13
13
10
13
10
3
13
10
13
3
8
13
3
7
10
3
10
3
13
3
3
13
13
13
3
10
3
3
3
13
13
13
13
9
13
13
3
13
3
13
13
3
13
3
13
13
10
13
10
10
13
10
3
3
13
3
7
10
13
3
13
3
13
10
3
13
3
3
3
13
3
12
3
13
6
3
3
3
3
10
13
13
14
3
13
3
3
10
3
7
3
13
3
13
10
3
13
10
13
3
13
3
13
3
3
3
13
3
13
10
3
13
13
13
3
13
3
3
3
3
13
13
3
13
3
3
13
13
13
3
13
13
3
3
3
13
3
13
3
3
3
3
10
13
13
13
3
3
13
13
13
3
13
3
13
13
3
7
3
13
3
12
13
13
3
3
3
3
3
13
3
13
3
3
3
3
3
13
13
3
3
13
3
3
3